In [31]:
from tensorflow.keras.layers import Layer
import numpy as np
import tensorflow as tf
import os

In [32]:
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        print(tf.math.abs(input_embedding - validation_embedding))
        return tf.math.abs(input_embedding - validation_embedding)

In [33]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [34]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [35]:
siamese_model = tf.keras.models.load_model('siamesemodelv2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

Tensor("l1_dist_2/Abs:0", shape=(None, 4096), dtype=float32)


In [36]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validation_img[0][

In [37]:
for image in os.listdir(os.path.join('application_data', 'verification_images')):
    validation_img = os.path.join('application_data', 'verification_images', image)
    print(validation_img)

application_data/verification_images/274239561_4997717153600513_1402768134350199826_n.jpg
application_data/verification_images/274307756_706026224102736_4223143425807525808_n.jpg
application_data/verification_images/274199022_653723539080066_3793464867576519753_n.jpg
application_data/verification_images/274162961_1178717622879582_1278848787476552094_n.jpg
application_data/verification_images/274138163_5152237061464923_3222415869394439790_n.jpg
application_data/verification_images/274362023_1313786832439451_7472318140337909848_n.jpg
application_data/verification_images/274196331_643731246890826_6136851217153407652_n.jpg
application_data/verification_images/274297736_320919406662996_8259553136556767358_n.jpg
application_data/verification_images/274199922_726588348333621_7236536409233883049_n.jpg
application_data/verification_images/274138999_633345184389091_5852268567621193232_n.jpg
application_data/verification_images/274301303_360289179280617_8481176449567133057_n.jpg
application_data/

In [38]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [39]:
import cv2
cap = cv2.VideoCapture(2)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 

        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(siamese_model, 0.5, 0.5)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Tensor("SiameseNetwork/l1_dist_2/Abs:0", shape=(None, 4096), dtype=float32)
False
True


In [40]:
np.sum(np.squeeze(results) > 0.9)

12

In [41]:
results

[array([[0.3440556]], dtype=float32),
 array([[0.31492126]], dtype=float32),
 array([[0.4516824]], dtype=float32),
 array([[0.61319715]], dtype=float32),
 array([[0.9658416]], dtype=float32),
 array([[0.97146714]], dtype=float32),
 array([[0.9568249]], dtype=float32),
 array([[0.9999832]], dtype=float32),
 array([[0.99999356]], dtype=float32),
 array([[0.9999573]], dtype=float32),
 array([[0.03656498]], dtype=float32),
 array([[0.94895875]], dtype=float32),
 array([[0.9764689]], dtype=float32),
 array([[0.9999927]], dtype=float32),
 array([[0.99984044]], dtype=float32),
 array([[0.64114356]], dtype=float32),
 array([[0.99997526]], dtype=float32),
 array([[0.99236745]], dtype=float32)]